In [120]:
import openpyxl
from itertools import islice

# 엑셀파일 열기
file = openpyxl.load_workbook('items.xlsx')

# sheet 가져오기
sett_sheet = file['set']
head_sheet = file['head']
body_sheet = file['body']
gloves_sheet = file['gloves']
belt_sheet = file['belt']
shoes_sheet = file['shoes']

# 장비 list 가져오기
HEAD, BODY, GLOVES, BELT, SHOES = [], [], [], [], []
SETS = dict()

# 아이템 가져오기 (이름, 힘, 민, 생, 지, 타저, 마저, 추공, 부위, 셋트, 활성여부)
for row in islice(head_sheet.values, 1, None):
    if row[10]: # 활성여부
        HEAD.append([x if x else 0 for x in row])
    
for row in islice(body_sheet.values, 1, None):
    if row[10]: # 활성여부
        BODY.append([x if x else 0 for x in row])
    
for row in islice(gloves_sheet.values, 1, None):
    if row[10]: # 활성여부
        GLOVES.append([x if x else 0 for x in row])
    
for row in islice(belt_sheet.values, 1, None):
    if row[10]: # 활성여부    
        BELT.append([x if x else 0 for x in row])
    
for row in islice(shoes_sheet.values, 1, None):
    if row[10]: # 활성여부
        SHOES.append([x if x else 0 for x in row])

# set효과 가져오기
for row in islice(sett_sheet.values, 1, None):
    row = [x if x else 0 for x in row]
    sett, num, HIM, MIN, SAENG, JI, TA, MA = row
    
    if sett in SETS:
        SETS[sett][num] = (HIM, MIN, SAENG, JI, TA, MA)
    else:
        SETS[sett] = dict()
        SETS[sett][num] = (HIM, MIN, SAENG, JI, TA, MA)

In [121]:
# 모든 방어구 조합 생성
from itertools import product
def get_all_comb():
    return product(HEAD, BODY, GLOVES, BELT, SHOES)

In [122]:
# 조합의 스텟 총합 계산
# 활성화된 SET효과도 표시
def get_total_stats(comb):
    HIM, MIN, SAENG, JI, TA, MA, CHU, SETT, NAMES = 0, 0, 0, 0, 0, 0, 0, {}, []
    # 아이템 개별 스탯 적용
    for item in comb:
        name, him, minn, saeng, ji, ta, ma, chu, part, sett, inactive = item
        NAMES.append(name)
        HIM += him
        MIN += minn
        SAENG += saeng
        JI += ji
        TA += ta
        MA += ma
        CHU += chu
        if sett in SETT:
            SETT[sett] += 1
        else:
            SETT[sett] = 1
            
    # 아이템 셋트효과 적용
    for sett in SETT:
        # 수련용 수룡왕 갑투 등은 셋트효과가 없음 => skip
        if sett not in SETS:
            continue

        num = SETT[sett]
        for i in range(1, num+1):
            if i in SETS[sett]:
                him, minn, saeng, ji, ta, ma = SETS[sett][i]
                HIM += him
                MIN += minn
                SAENG += saeng
                JI += ji
                TA += ta
                MA += ma
                
    return (HIM, MIN, SAENG, JI, TA, MA, CHU, NAMES)

In [135]:
def print_result(result):
    stats = ['힘', '민', '생', '지', '타저', '마저', '추공', '아이템']
    header = ''
    for stat in stats:
        header += stat.ljust(4)
    print(header)
    for row in result:
        row_str = ''
        for stat in row:
            row_str += str(stat).rjust(4)
            
        print(row_str)

In [136]:
# 스텟셋팅
results = []
for comb in get_all_comb():
    stat = list(get_total_stats(comb))
    results.append(stat)

# 힘 민 생 지 타저 마저 추공
results.sort(key = lambda x: (x[0], x[3]), reverse = True) # (힘, 지)
print('total', len(results), 'combinations')
print_result(results[:])

total 10125 combinations
힘   민   생   지   타저  마저  추공  아이템 
1225   0 350 125 145 110   0['브리트라투구', '백호피갑', '백호손목보호대', '황제의요대', '황제의신발']
1200   0 325 125 160 110   0['백호의머리띠', '백호피갑', '백호손목보호대', '황제의요대', '백호의전투화']
1200   0 325 125 150 105   0['브리트라투구', '백호피갑', '황제의장갑', '황제의요대', '백호의전투화']
1175   0 500 175 155 105   0['백호의머리띠', '백호피갑', '황제의장갑', '황제의요대', '황제의신발']
1175   0 375 150 160 110   0['백호의머리띠', '백호피갑', '백호손목보호대', '황제의요대', '황제의신발']
1175   0 475 150 150 105   0['브리트라투구', '백호피갑', '황제의장갑', '황제의요대', '황제의신발']
1175   0 250 100 145 110   0['브리트라투구', '백호피갑', '백호손목보호대', '백호의요대', '백호의전투화']
1150   0 375 175 145 115   0['백호의머리띠', '백호피갑', '황제의장갑', '황제의요대', '파멸자의전투화']
1150   0 225 175 160 110   0['백호의머리띠', '백호피갑', '백호손목보호대', '파멸자의요대', '백호의전투화']
1150   0 350 150 165 105   0['백호의머리띠', '백호피갑', '황제의장갑', '황제의요대', '백호의전투화']
1150   0 325 150 150 110   0['백호의머리띠', '백호피갑', '백호손목보호대', '백호의요대', '황제의신발']
1150   0 350 150 140 115   0['브리트라투구', '백호피갑', '황제의장갑', '황제의요대', '파멸자의전투화']
1150   0 325 150 140 105   0['브리

 800   0 150 289 135  85   0['해왕투구', '백호피갑', '파멸자의장갑', '파멸자의요대', '백호의전투화']
 800   0 500 275 125 125   0['백호의머리띠', '현무피갑', '치우천왕수', '현무의요대', '파멸자의전투화']
 800   0 500 275 135 115   0['백호의머리띠', '현무피갑', '치우천왕수', '백호의요대', '현무의전투화']
 800   0 500 275 145 115   0['백호의머리띠', '개마갑주', '백호손목보호대', '황제의요대', '현무의전투화']
 800   0 475 275 135 115   0['백호의머리띠', '개마갑주', '백호손목보호대', '현무의요대', '황제의신발']
 800   0 425 275 145 115   0['백호의머리띠', '개마갑주', '백호손목보호대', '백호의요대', '황제의신발']
 800   0 300 275 120 140   0['백호의머리띠', '파멸자갑옷', '파멸자의장갑', '태황의요대', '파멸자의전투화']
 800 100 350 275 145 110  30['백호의머리띠', '고구려태황갑옷', '황제의장갑', '파멸자의요대', '현무의전투화']
 800 100 375 275 120 125  30['백호의머리띠', '고구려태황갑옷', '치우천왕수', '백호의요대', '파멸자의전투화']
 800 100 325 275 160 100   0['고대금관', '현무피갑', '백호손목보호대', '황제의요대', '태황의신발']
 800  50 400 275 150 100   0['고대금관', '백호피갑', '태황의장갑', '황제의요대', '현무의전투화']
 800  50 375 275 140 100   0['고대금관', '백호피갑', '태황의장갑', '현무의요대', '황제의신발']
 800   0 250 275 135 120   0['고대금관', '파멸자갑옷', '백호손목보호대', '태황의요대', '백호의전투화']
 800   0 400 2

 725   0 275 190 125  85   0['수련용수룡왕투구', '백호피갑', '파멸자의장갑', '백호의요대', '현무의전투화']
 725  50 225 190 130  85   0['수련용수룡왕투구', '백호피갑', '태황의장갑', '파멸자의요대', '황제의신발']
 725   0 275 190 120 100   0['수련용수룡왕투구', '파멸자갑옷', '황제의장갑', '태황의요대', '황제의신발']
 725   0 200 190 135  75   0['수련용수룡왕투구', '파멸자갑옷', '백호손목보호대', '현무의요대', '백호의전투화']
 725   0 150 190 145  75   0['수련용수룡왕투구', '파멸자갑옷', '백호손목보호대', '백호의요대', '백호의전투화']
 725   0 375 189 105 120   0['해왕투구', '현무피갑', '황제의장갑', '태황의요대', '황제의신발']
 725   0 350 189 120  95   0['해왕투구', '현무피갑', '백호손목보호대', '현무의요대', '백호의전투화']
 725   0 250 189 130  95   0['해왕투구', '현무피갑', '백호손목보호대', '백호의요대', '백호의전투화']
 725   0 400 189 130  85   0['해왕투구', '백호피갑', '황제의장갑', '현무의요대', '현무의전투화']
 725 100 275 189 145  80   0['해왕투구', '백호피갑', '치우천왕수', '황제의요대', '태황의신발']
 725 100 400 175 125 135   0['백호의머리띠', '현무피갑', '치우천왕수', '태황의요대', '태황의신발']
 725 150 275 175 155  80   0['백호의머리띠', '파멸자갑옷', '태황의장갑', '현무의요대', '태황의신발']
 725   0 250 175 125 100   0['백호의머리띠', '해왕갑옷', '황제의장갑', '현무의요대', '백호의전투화']
 725   0 200 175 

 650 100 100 290 160  60   0['수련용수룡왕투구', '파멸자갑옷', '황제의장갑', '파멸자의요대', '태황의신발']
 650 100 200 290 115 105  30['수련용수룡왕투구', '고구려태황갑옷', '백호손목보호대', '파멸자의요대', '파멸자의전투화']
 650 100 300 290 115 100  30['수련용수룡왕투구', '고구려태황갑옷', '파멸자의장갑', '황제의요대', '파멸자의전투화']
 650   0 350 289 135  90   0['해왕투구', '현무피갑', '황제의장갑', '파멸자의요대', '현무의전투화']
 650   0 375 289 135  90   0['해왕투구', '개마갑주', '황제의장갑', '황제의요대', '백호의전투화']
 650   0 200 289 135  75   0['해왕투구', '파멸자갑옷', '치우천왕수', '백호의요대', '백호의전투화']
 650 100  75 289 155  65   0['해왕투구', '파멸자갑옷', '백호손목보호대', '파멸자의요대', '태황의신발']
 650 100 175 289 155  60   0['해왕투구', '파멸자갑옷', '파멸자의장갑', '황제의요대', '태황의신발']
 650  50 250 289 130  80   0['해왕투구', '파멸자갑옷', '태황의장갑', '황제의요대', '파멸자의전투화']
 650  50 525 275 140 110   0['백호의머리띠', '개마갑주', '태황의장갑', '황제의요대', '현무의전투화']
 650  50 500 275 130 110   0['백호의머리띠', '개마갑주', '태황의장갑', '현무의요대', '황제의신발']
 650 100 325 275 130 125   0['고대금관', '현무피갑', '파멸자의장갑', '태황의요대', '태황의신발']
 650  50 400 275 105 145   0['고대금관', '현무피갑', '태황의장갑', '태황의요대', '파멸자의전투화']
 650  50 275 2

 600 100 375 150 110 130  30['악령면', '고구려태황갑옷', '황제의장갑', '태황의요대', '현무의전투화']
 600 200 275 150 120 120  30['악령면', '고구려태황갑옷', '황제의장갑', '태황의요대', '태황의신발']
 600 100 275 150 105 130  30['악령면', '고구려태황갑옷', '파멸자의장갑', '태황의요대', '백호의전투화']
 600   0 175 150 115 105   0['악령면', '해왕갑옷', '백호손목보호대', '황제의요대', '파멸자의전투화']
 600   0 150 150 115  95   0['악령면', '해왕갑옷', '백호손목보호대', '백호의요대', '황제의신발']
 600  50 425 145 135  90   0['백호의머리띠', '봉황갑옷', '태황의장갑', '황제의요대', '현무의전투화']
 600  50 400 145 125  90   0['백호의머리띠', '봉황갑옷', '태황의장갑', '현무의요대', '황제의신발']
 600  50 325 145  95  70   0['백호의머리띠', '수련용수룡왕갑옷', '태황의장갑', '황제의요대', '현무의전투화']
 600  50 300 145  85  70   0['백호의머리띠', '수련용수룡왕갑옷', '태황의장갑', '현무의요대', '황제의신발']
 600 100 325 145 135  85   0['브리트라투구', '봉황갑옷', '치우천왕수', '현무의요대', '태황의신발']
 600 100 275 145 135  85   0['브리트라투구', '봉황갑옷', '치우천왕수', '백호의요대', '태황의신발']
 600 100 225 145  95  65   0['브리트라투구', '수련용수룡왕갑옷', '치우천왕수', '현무의요대', '태황의신발']
 600 100 175 145  95  65   0['브리트라투구', '수련용수룡왕갑옷', '치우천왕수', '백호의요대', '태황의신발']
 600  50 325 145 

 550   0 225 220  65 110   0['파멸자투구', '수련용수룡왕갑옷', '파멸자의장갑', '태황의요대', '파멸자의전투화']
 550 200 225 220 180  50  30['고구려태황투구', '봉황갑옷', '황제의장갑', '파멸자의요대', '태황의신발']
 550 200 125 220 140  30  30['고구려태황투구', '수련용수룡왕갑옷', '황제의장갑', '파멸자의요대', '태황의신발']
 550 100 435 220 150  75   0['봉황투구', '현무피갑', '치우천왕수', '현무의요대', '태황의신발']
 550 100 335 220 150  75   0['봉황투구', '현무피갑', '치우천왕수', '백호의요대', '태황의신발']
 550 100 385 220 160  75   0['봉황투구', '개마갑주', '백호손목보호대', '황제의요대', '태황의신발']
 550 100 235 220 145  80   0['봉황투구', '파멸자갑옷', '파멸자의장갑', '태황의요대', '태황의신발']
 550  50 310 220 120 100   0['봉황투구', '파멸자갑옷', '태황의장갑', '태황의요대', '파멸자의전투화']
 550 100 435 220 115 115  30['봉황투구', '고구려태황갑옷', '치우천왕수', '태황의요대', '황제의신발']
 550 150 260 220 140  80  30['봉황투구', '고구려태황갑옷', '태황의장갑', '파멸자의요대', '백호의전투화']
 550   0 175 220 125  95   0['악령면', '봉황갑옷', '백호손목보호대', '파멸자의요대', '파멸자의전투화']
 550   0 275 220 125  90   0['악령면', '봉황갑옷', '파멸자의장갑', '황제의요대', '파멸자의전투화']
 550   0  75 220  85  75   0['악령면', '수련용수룡왕갑옷', '백호손목보호대', '파멸자의요대', '파멸자의전투화']
 550   0 175 22

 475 150 400 195 130  90  30['고구려태황투구', '봉황갑옷', '태황의장갑', '태황의요대', '황제의신발']
 475 100 275 195  95  75  30['고구려태황투구', '수련용수룡왕갑옷', '치우천왕수', '태황의요대', '백호의전투화']
 475 200 200 195 125  30  30['고구려태황투구', '수련용수룡왕갑옷', '파멸자의장갑', '현무의요대', '태황의신발']
 475 200 150 195 125  30  30['고구려태황투구', '수련용수룡왕갑옷', '파멸자의장갑', '백호의요대', '태황의신발']
 475 150 275 195 100  50  30['고구려태황투구', '수련용수룡왕갑옷', '태황의장갑', '현무의요대', '파멸자의전투화']
 475 150 225 195 100  50  30['고구려태황투구', '수련용수룡왕갑옷', '태황의장갑', '백호의요대', '파멸자의전투화']
 475 150 300 195  90  70  30['고구려태황투구', '수련용수룡왕갑옷', '태황의장갑', '태황의요대', '황제의신발']
 475 150 485 195 130  80  30['봉황투구', '고구려태황갑옷', '태황의장갑', '현무의요대', '현무의전투화']
 475 250 335 195 140  70  30['봉황투구', '고구려태황갑옷', '태황의장갑', '현무의요대', '태황의신발']
 475 150 385 195 130  80  30['봉황투구', '고구려태황갑옷', '태황의장갑', '백호의요대', '현무의전투화']
 475 250 285 195 140  70  30['봉황투구', '고구려태황갑옷', '태황의장갑', '백호의요대', '태황의신발']
 475   0 260 195 120  80   0['봉황투구', '해왕갑옷', '황제의장갑', '현무의요대', '파멸자의전투화']
 475   0 210 195 120  80   0['봉황투구', '해왕갑옷', '황제의장갑', '백호의요대', '파멸자의

 400   0 360 115 115  90   0['봉황투구', '봉황갑옷', '파멸자의장갑', '태황의요대', '백호의전투화']
 400   0 310 115  80  70   0['봉황투구', '수련용수룡왕갑옷', '황제의장갑', '태황의요대', '현무의전투화']
 400   0 210 115  75  70   0['봉황투구', '수련용수룡왕갑옷', '파멸자의장갑', '태황의요대', '백호의전투화']
 400   0 225 115  85 115   0['수련용수룡왕투구', '해왕갑옷', '백호손목보호대', '태황의요대', '현무의전투화']
 400 150 150 115 120  70   0['수련용수룡왕투구', '해왕갑옷', '태황의장갑', '황제의요대', '태황의신발']
 400   0 350 110 100 100   0['수련용수룡왕투구', '봉황갑옷', '황제의장갑', '태황의요대', '현무의전투화']
 400   0 250 110  95 100   0['수련용수룡왕투구', '봉황갑옷', '파멸자의장갑', '태황의요대', '백호의전투화']
 400   0 250 110  60  80   0['수련용수룡왕투구', '수련용수룡왕갑옷', '황제의장갑', '태황의요대', '현무의전투화']
 400   0 150 110  55  80   0['수련용수룡왕투구', '수련용수룡왕갑옷', '파멸자의장갑', '태황의요대', '백호의전투화']
 400   0 325 109  95 105   0['해왕투구', '봉황갑옷', '백호손목보호대', '태황의요대', '현무의전투화']
 400 150 250 109 130  60   0['해왕투구', '봉황갑옷', '태황의장갑', '황제의요대', '태황의신발']
 400   0 225 109  55  85   0['해왕투구', '수련용수룡왕갑옷', '백호손목보호대', '태황의요대', '현무의전투화']
 400 150 150 109  90  40   0['해왕투구', '수련용수룡왕갑옷', '태황의장갑', '황제의요대', '태황의신

In [ ]:
# 타저 마저 셋팅
character_hit = int(input('기본 타저'))
character_magic = int(input('기본 마저'))
min_hit = int(input('최소타저'))
min_magic = int(input('최소마저'))
results = []

for comb in get_all_comb():
    stat = list(get_total_stats(comb))
    TA = stat[4] + character_hit # 타저
    MA = stat[5] + character_magic # 마저
    if TA >= min_hit and MA >= min_magic:
        stat[4] = TA # 기본타저 적용
        stat[5] = MA # 기본마저 적용
        results.append(stat)
        
results.sort(key = lambda x: (x[4], x[5]), reverse=True) # (타저,마저) asc
print('total', len(results), 'combinations')
print_result(results[:])